<a href="https://colab.research.google.com/github/Areej17-01/Generative-AI-LLMs-and-Transformers./blob/main/Lora_training_with_llama_1b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Feb  9 16:25:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:


import torch
# Check so there is a gpu available, a T4(free tier) is enough to run this notebook
assert (torch.cuda.is_available()==True)


!pip  install --no-build-isolation axolotl[flash-attn,deepspeed]

In [3]:
!pip install -e 'git+https://github.com/axolotl-ai-cloud/axolotl.git@78b42a3fe13c49e317bc116b9999c30e070322cc#egg=axolotl' # ensures the same version we used in the course

Obtaining axolotl from git+https://github.com/axolotl-ai-cloud/axolotl.git@78b42a3fe13c49e317bc116b9999c30e070322cc#egg=axolotl
  Cloning https://github.com/axolotl-ai-cloud/axolotl.git (to revision 78b42a3fe13c49e317bc116b9999c30e070322cc) to ./src/axolotl
  Running command git clone --filter=blob:none --quiet https://github.com/axolotl-ai-cloud/axolotl.git /content/src/axolotl
  Running command git rev-parse -q --verify 'sha^78b42a3fe13c49e317bc116b9999c30e070322cc'
  Running command git fetch -q https://github.com/axolotl-ai-cloud/axolotl.git 78b42a3fe13c49e317bc116b9999c30e070322cc
  Running command git checkout -q 78b42a3fe13c49e317bc116b9999c30e070322cc
  Resolved https://github.com/axolotl-ai-cloud/axolotl.git to commit 78b42a3fe13c49e317bc116b9999c30e070322cc
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/huggingface/transformers.git (to revision 026a173a64372e9602a16523b8fae9de4b0ff428) to /tmp/pip-install-_5lb2knt/transformers_f85f05ef7dbd4c159a3b0ad268

In [7]:
import yaml

train_config = """
# model params
base_model: unsloth/Llama-3.2-1B-Instruct

# dataset params
datasets:
  - path: jaydenccc/AI_Storyteller_Dataset
    type:
      system_prompt: "You are an amazing storyteller. From the following synopsis, create an engaging story."
      field_system: system
      field_instruction: synopsis
      field_output: short_story
      format: "<|user|>\n {instruction} </s>\n<|assistant|>"
      no_input_format: "<|user|> {instruction} </s>\n<|assistant|>"

output_dir: ./models/Llama3_Storyteller2


# model params
sequence_length: 512
bf16: auto
tf32: false

# training params
micro_batch_size: 1
num_epochs: 1
optimizer: adamw_bnb_8bit
learning_rate: 0.0001

logging_steps: 1


# LoRA
adapter: lora

lora_r: 16
lora_alpha: 16
lora_dropout: 0.05

lora_target_linear: true

# Gradient Accumulation
gradient_accumulation_steps: 1

# Gradient Checkpointing
gradient_checkpointing: true
"""

# Convert the YAML string to a Python dictionary
yaml_dict = yaml.safe_load(train_config)


# Write the YAML file
with open("advanced_train.yml", 'w') as file:
    yaml.dump(yaml_dict, file)


In [5]:
!pip show transformers


Name: transformers
Version: 4.48.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: axolotl, lm_eval, optimum, peft, sentence-transformers, trl


In [6]:
!pip show axolotl


Name: axolotl
Version: 0.6.0
Summary: LLM Trainer
Home-page: https://axolotl-ai-cloud.github.io/axolotl/
Author: 
Author-email: 
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: accelerate, addict, antlr4-python3-runtime, art, bitsandbytes, colorama, datasets, einops, evaluate, fastcore, fire, gcsfs, gradio, hf-transfer, immutabledict, langdetect, liger-kernel, lm-eval, numba, numpy, nvidia-ml-py, optimum, packaging, peft, pydantic, python-dotenv, PyYAML, requests, s3fs, schedulefree, scikit-learn, scipy, sentencepiece, tensorboard, tokenizers, torch, torchao, transformers, triton, trl, wandb, xformers, zstandard
Required-by: 


In [1]:
!pip install transformers==4.47.1 --force-reinstall
# !pip install --upgrade transformers


  Using cached transformers-4.47.1-py3-none-any.whl.metadata (44 kB)
  Using cached filelock-3.17.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached huggingface_hub-0.28.1-py3-none-any.whl.metadata (13 kB)
  Using cached numpy-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached regex-2024.11.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached safetensors-0.5.2-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached fsspec-2025.2.0-py3-none-any.whl.metadata (11 kB)
  Usi

In [2]:
!pip install numpy==2.1 --force-reinstall


  Using cached numpy-2.1.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached numpy-2.1.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.2
    Uninstalling numpy-2.2.2:
      Successfully uninstalled numpy-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 3.50.2 requires numpy~=1.0, but you have numpy 2.1.0 which is incompatible.
axolotl 0.6.0 requires numpy<=2.0.1,>=1.24.4, but you have numpy 2.1.0 which is incompatible.
axolotl 0.6.0 requires packaging==23.2, but you have packaging 24.2 which is incompatible.
datasets 3.1.0 requires fsspec[http]<=2024.9.0,>=2023.1.0, but you have fsspec 2025.2.0 which is incompatible.
albumentations 2.0.3 requires pydantic>=2.9.2, but you have pydantic 2.6.3 which is incompatible.
lan

In [8]:
!accelerate launch -m axolotl.cli.train advanced_train.yml

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-02-09 16:47:40.784223: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739119660.812439    9825 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739119660.822585    9825 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-09 16:47:40.856933: I tensorflow/core/platform

In [9]:
# Optional: Merge the trained adapter
!accelerate launch -m axolotl.cli.merge_lora advanced_train.yml

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-02-09 16:52:41.294660: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739119961.328811   11156 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739119961.338883   11156 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-09 16:52:41.372016: I tensorflow/core/platform

In [10]:
from transformers import pipeline

In [14]:
from transformers import pipeline
import torch


In [15]:
pipe = pipeline("text-generation", model="/content/models/Llama3_Storyteller2/merged", torch_dtype=torch.bfloat16, device_map="auto")

Device set to use cuda:0


In [31]:
messages = [
    {"role":"system", "content": "You are an amazing storyteller. From the following synopsis, create an engaging story."},
    {"role": "user", "content": "A woman discovers a hidden talent for cooking and starts her own successful restaurant."},
]


In [32]:
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [33]:
outputs = pipe(prompt, max_new_tokens=128)

print(outputs[0]["generated_text"])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 09 Feb 2025

You are an amazing storyteller. From the following synopsis, create an engaging story.<|eot_id|><|start_header_id|>user<|end_header_id|>

A woman discovers a hidden talent for cooking and starts her own successful restaurant.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

As a child, Emma had always been fascinated by the way her mother could transform simple ingredients into culinary masterpieces. She would watch in awe as she sautéed vegetables, roasted meats, and baked bread, the aromas wafting from the kitchen like a warm hug. But it wasn't until her mother passed away that Emma realized her true passion lay in cooking.

One day, while rummaging through her mother's old recipe books, Emma stumbled upon a recipe for a traditional Italian dish she had never tried before: homemade ravioli. Intrigued, she decided to give it a go, and to her surprise


In [25]:
hg_pipe = pipeline("text-generation", model="unsloth/Llama-3.2-1B-Instruct", torch_dtype=torch.bfloat16, device_map="auto")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


In [26]:
prompt = hg_pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [30]:
outputs = hg_pipe(prompt, max_new_tokens=128)

print(outputs[0]["generated_text"])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 09 Feb 2025

You are an amazing storyteller. From the following synopsis, create an engaging story.<|eot_id|><|start_header_id|>user<|end_header_id|>

a person died in crash while driving home<|eot_id|><|start_header_id|>assistant<|end_header_id|>

As the sun began to set on a typical Friday evening, John had just finished a long week of work at the office. He had been looking forward to a relaxing night at home, but fate had other plans. As he pulled out of the driveway, his eyes were fixed on the road ahead, his hands gripping the steering wheel tightly as he navigated through the crowded streets.

It was a typical Friday evening, with people rushing to get home from work and school, and John was just trying to get to his own house without any major incidents. But as he approached the intersection of Elm and Oak, he failed to notice a pedestrian crossing


In [35]:
messages = [
    {"role":"system", "content": "You are an amazing storyteller. From the following synopsis, create an engaging story in urdu language."},
    {"role": "user", "content": "A woman discovers a hidden talent for cooking and starts her own successful restaurant."},
]
prompt = hg_pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = hg_pipe(prompt, max_new_tokens=128)

print(outputs[0]["generated_text"])


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 09 Feb 2025

You are an amazing storyteller. From the following synopsis, create an engaging story in urdu language.<|eot_id|><|start_header_id|>user<|end_header_id|>

A woman discovers a hidden talent for cooking and starts her own successful restaurant.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

مریم نئی سے ایک خاتون نے ایک مشہور رستہ کھانے کی چھوٹی چھوٹی کھانے کی کھانے کی کوشش کی۔

مریم کی پہلی بار اس کی پیدائش ایک گھر میں ہوئی تھی جہاں اس نے اپنے والدین سے کھانے کی کھانے کی کہانی سنی
